# Preparating Data

In [1]:
import pandas as pd
import numpy as np
import json


In [2]:
# import variables of all process
sample_frac = 0.25
frac_nan_values = 0.05

#some important dicts
map_D63 = dict(zip(['CR', 'CO', 'CL', 'XL', 'XZ', 'XM'], range(6)))
map_D64 = dict(zip(['O', 'R', np.nan, 'U', '-1'], [0, 1, np.nan, 2, 3]))
columns_dtype = {'B_31' : 'int8', **json.load(open('columns_map.json'))}
columns_to_drop = ['S_3', 'D_42', 'D_43', 'D_46', 'D_48', 'D_49', 'D_50', 'D_53',
                   'S_7', 'D_56', 'S_9', 'D_62', 'B_17', 'D_66', 'D_73', 'D_76',
                   'D_77', 'R_9', 'D_82', 'B_29', 'D_87', 'D_88', 'D_105', 'D_106',
                   'R_26', 'R_27', 'D_108', 'D_110', 'D_111', 'B_39', 'S_27', 'B_42',
                   'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142','S_2']


# Reading data in chunks
all_columns = list(pd.read_csv('C:/Users/DavidG/Documents/american_express_data/train_data.csv', nrows =1))
columns_to_read = np.setdiff1d(all_columns, columns_to_drop)
train_result = pd.read_csv('C:/Users/DavidG/Documents/american_express_data/train_labels.csv', dtype = {'target': 'int8'}) # Read the result of train data

train_partitions = np.array_split(train_result.sample(frac = 1, random_state = 12345), int(1/sample_frac))


#print("Fraction of 1 in the complete dataset is {}".format(sum(train_result['target'])/len(train_result))) # Showing the fraction of 1 in the all data
#print("Fraction of 1 in the sample dataset is {}".format(sum(train_result_sample['target'])/len(train_result_sample))) # Showing the fraction of 1 in the sample data

In [3]:
n = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
div_val = n*((n*n).cumsum()) - (n.cumsum())**2
div = dict(zip(n, div_val))
div = {0:np.nan, **div}
div[1] = np.nan
sum_val = dict(zip(n, n.cumsum()))
sum_val = {0:np.nan, **sum_val}

for k, partition in enumerate(train_partitions):
    train_data = pd.read_csv('C:/Users/DavidG/Documents/american_express_data/train_data.csv', chunksize = 500000, dtype = columns_dtype, usecols = columns_to_read)
    train_result_sample = partition
    chunk_list = []
    for chunk in train_data:
        chunk_list.append(pd.merge(train_result_sample, chunk, how = 'inner', on = 'customer_ID'))

    train_data_sample = pd.concat(chunk_list)


    del chunk_list
    print('Ready the lecture of {} partition'.format(k))
    
    
    train_data_sample['D_63'] = train_data_sample.apply(lambda x: map_D63[x['D_63']], axis = 1)
    train_data_sample['D_64'] = train_data_sample.apply(lambda x: map_D64[x['D_64']], axis = 1)
    print('Ready apply of {} partition'.format(k))
    
    train_data_sample.reset_index(drop = True, inplace = True)
    train_data_sample['n'] = np.ones(len(train_data_sample), dtype = 'int8')
    j = 1
    for i in range(1, len(train_data_sample)):
        if train_data_sample.loc[(i-1, 'customer_ID')] == train_data_sample.loc[(i, 'customer_ID')]:
            j += 1
        else:
            j = 1
        train_data_sample.loc[(i, 'n')] = j
    cont_pd = train_data_sample.drop(['target', 'n'], axis = 1).groupby('customer_ID').count()
    print('Ready add n values of {} partition'.format(k))
    
    # m values
    m_values = train_data_sample.drop(['target', 'n', 'customer_ID'], axis = 1).multiply(train_data_sample['n'], axis = 0)
    m_values = m_values.astype('float32')
    m_values = pd.concat([m_values, train_data_sample['customer_ID']], axis = 1)
    m_values = m_values.groupby('customer_ID').sum() * cont_pd
    m_values = m_values + cont_pd.applymap(lambda x: -sum_val[x])*train_data_sample.drop(['target', 'n'], axis = 1).groupby('customer_ID').prod()
    m_values = m_values.astype('float32')
    m_values = m_values / cont_pd.applymap(lambda x: div[x])
    m_values = m_values.astype('float32')
    print('Ready m_values of {} partition'.format(k))
    
    # Other features
    X_train = train_data_sample.drop(['target'], axis = 1).groupby('customer_ID').agg([np.mean, np.max, np.min, np.std])
    X_train = pd.DataFrame(data = X_train.values.astype('float32'), index = X_train.index, columns = [x[0] + '_' + x[1] for x in X_train.columns])
    X_train = pd.merge(X_train, m_values, how = 'inner', left_index = True, right_index = True)
    final_data_dropna = pd.merge(X_train.dropna(), train_result_sample, on = 'customer_ID', how = 'inner')
    final_data_fillna = pd.merge(X_train.fillna(X_train.mean()), train_result_sample, on = 'customer_ID', how = 'inner')
    final_data_dropna.to_parquet('C:/Users/DavidG/Documents/american_express_data/dropna_data_{}.parquet'.format(k))
    final_data_fillna.to_parquet('C:/Users/DavidG/Documents/american_express_data/fillna_data_{}.parquet'.format(k))
    print('Ready other features of {} partition and save'.format(k))
    
    # Eliminate variables
    del m_values
    del X_train
    del train_data_sample
    del train_result_sample
    del cont_pd
    del final_data_dropna
    del final_data_fillna

Ready the lecture of 0 partition
Ready apply of 0 partition
Ready add n values of 0 partition
Ready m_values of 0 partition
Ready other features of 0 partition and save
Ready the lecture of 1 partition
Ready apply of 1 partition
Ready add n values of 1 partition
Ready m_values of 1 partition
Ready other features of 1 partition and save
Ready the lecture of 2 partition
Ready apply of 2 partition
Ready add n values of 2 partition
Ready m_values of 2 partition
Ready other features of 2 partition and save
Ready the lecture of 3 partition
Ready apply of 3 partition
Ready add n values of 3 partition
Ready m_values of 3 partition
Ready other features of 3 partition and save
